# ニューラルネットワークとnosetests（改訂版）

プロダクションのテストケースが、ニューラルネットワークでどの程度通るかを試してみました。

すべてのテストケースが通るように、ニューラルネットワークに関する処理をプロダクションコードに追加しています。

## (1) コードを修正

現行のロジスティック回帰を、ニューラルネットワーク（MLP）に変更します。

また、ニューラルネットワークにより学習／予測が行われるよう、テスト条件を修正します。

### (1-1) 修正を行ったプロダクションコード：

### learning_parameter.py

（定数定義 ALGORITHM_NEURAL_NETWORK = 2 を追加）

```
class LearningParameter:
    :
    ALGORITHM_NEURAL_NETWORK = 2
```

### bot.py

（MLPClassifier により学習・評価を行う処理を追加）

```
from sklearn.neural_network import MLPClassifier
:
class Bot:
    def __init__(self, bot_id, learning_parameter):
        :
        elif self.learning_parameter.algorithm == LearningParameter.ALGORITHM_NEURAL_NETWORK:
            logger.debug('use algorithm: Multi-layer Perceptron')

            activation = self.learning_parameter.params_for_algorithm.get('activation', None)
            if activation is None:
                logger.debug('learning_parameter has not parameter [activation]')
                params = {'activation':['identity','logistic','tanh','relu']}
                grid = GridSearchCV(
                    MLPClassifier(max_iter=10000, shuffle=False, random_state=0),
                    param_grid=params
                )
                grid.fit(training_set.x, training_set.y)
                estimator = grid.best_estimator_
                logger.debug('best_params_: %s' % grid.best_params_)
            else:
                logger.debug('learning_parameter has parameter [activation]')
                estimator = MLPClassifier(
                    activation=activation, max_iter=10000, shuffle=False, random_state=0
                )
                estimator.fit(training_set.x, training_set.y)
            :
```

### (1-2) 修正を行ったテストコード：

### learning/learning/tests/helper.py

テスト条件を「ALGORITHM_LOGISTIC_REGRESSION」から「ALGORITHM_NEURAL_NETWORK」に変更しています。

```
def learning_parameter(
        :
        # algorithm=LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
        algorithm=LearningParameter.ALGORITHM_NEURAL_NETWORK,
        params_for_algorithm=None
    ):
    :
```

### (1-3) その他のコード：

一切変更しておりません。

## (2) テスト結果

コマンド「nosetests -s learning/tests」を実行しました。

全ケース中、下記の2ケース（ラベル０に対して回答不能）がエラーとなりました。


- test_fail_blank


- test_fail_want_to_eat_ramen

<b><a href="07-nosetests-checkpoint.ipynb">こちらのレポートで確認</a></b>したのですが、MLP ですと、ラベル０に対する proba が閾値より低くなってしまう問題があるようです。

これは、ラベル0と予測された場合と、低しきい値で予測された場合についての扱いを工夫することで、対処できるかと存じます。

```
MacBookPro-makmorit-jp:learning makmorit$ date
Tue Apr 11 14:55:28 JST 2017
MacBookPro-makmorit-jp:learning makmorit$ pwd
/Users/makmorit/GitHub/donusagi-bot/learning
MacBookPro-makmorit-jp:learning makmorit$ nosetests -s learning/tests
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)
1.0
{'database': {'password': '', 'endpoint': 'mysql://root@localhost/donusagi_bot_test?charset=utf8', 'host': 'localhost', 'name': 'donusagi_bot_test', 'user': 'root'}}
..........0.983495145631
{'database': {'password': '', 'endpoint': 'mysql://root@localhost/donusagi_bot_test?charset=utf8', 'host': 'localhost', 'name': 'donusagi_bot_test', 'user': 'root'}}
.{'database': {'password': '', 'endpoint': 'mysql://root@localhost/donusagi_bot_test?charset=utf8', 'host': 'localhost', 'name': 'donusagi_bot_test', 'user': 'root'}}
./Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/cross_validation.py:552: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of labels for any class cannot be less than n_folds=3.
  % (min_labels, self.n_folds)), Warning)
0.986474094452
{'database': {'password': '', 'endpoint': 'mysql://root@localhost/donusagi_bot_test?charset=utf8', 'host': 'localhost', 'name': 'donusagi_bot_test', 'user': 'root'}}
.{'database': {'password': '', 'endpoint': 'mysql://root@localhost/donusagi_bot_test?charset=utf8', 'host': 'localhost', 'name': 'donusagi_bot_test', 'user': 'root'}}
.{'database': {'password': '', 'endpoint': 'mysql://root@localhost/donusagi_bot_test?charset=utf8', 'host': 'localhost', 'name': 'donusagi_bot_test', 'user': 'root'}}
.{'database': {'password': '', 'endpoint': 'mysql://root@localhost/donusagi_bot_test?charset=utf8', 'host': 'localhost', 'name': 'donusagi_bot_test', 'user': 'root'}}
.0.97458369851
{'database': {'password': '', 'endpoint': 'mysql://root@localhost/donusagi_bot_test?charset=utf8', 'host': 'localhost', 'name': 'donusagi_bot_test', 'user': 'root'}}
F0.97458369851
{'database': {'password': '', 'endpoint': 'mysql://root@localhost/donusagi_bot_test?charset=utf8', 'host': 'localhost', 'name': 'donusagi_bot_test', 'user': 'root'}}
F0.97458369851
{'database': {'password': '', 'endpoint': 'mysql://root@localhost/donusagi_bot_test?charset=utf8', 'host': 'localhost', 'name': 'donusagi_bot_test', 'user': 'root'}}
.0.97458369851
{'database': {'password': '', 'endpoint': 'mysql://root@localhost/donusagi_bot_test?charset=utf8', 'host': 'localhost', 'name': 'donusagi_bot_test', 'user': 'root'}}
.0.97458369851
{'database': {'password': '', 'endpoint': 'mysql://root@localhost/donusagi_bot_test?charset=utf8', 'host': 'localhost', 'name': 'donusagi_bot_test', 'user': 'root'}}
..
======================================================================
FAIL: test_fail_blank (learning.tests.engine.test_ptna_conversation.PtnaConversationTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/makmorit/GitHub/donusagi-bot/learning/learning/tests/engine/test_ptna_conversation.py", line 66, in test_fail_blank
    ok_(result.probability > self.threshold)
AssertionError: None
-------------------- >> begin captured logging << --------------------
learning.log: DEBUG: learning_parameter: {'_include_failed_data': False, '_params_for_algorithm': {}, '_include_tag_vector': False, '_algorithm': 2, '_classify_threshold': None}
learning.log: DEBUG: start Bot#learn
learning.log: DEBUG: TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4559
learning.log: DEBUG: TextArray#__init__ start
learning.log: DEBUG: TextArray#to_vec start
learning.log: DEBUG: TextArray#to_vec end
learning.log: DEBUG: Bot#__build_training_set training_set.count_sample_by_y: [[0, 3], [2671, 226], [2673, 166], [2674, 32], [2675, 152], [2676, 130], [2677, 124], [2678, 72], [2679, 150], [2680, 78], [2681, 96], [2682, 66], [2683, 96], [2684, 46], [2685, 40], [2686, 52], [2687, 142], [2688, 142], [2689, 176], [2690, 66], [2691, 46], [2692, 156], [2694, 76], [2695, 72], [2696, 90], [2697, 142], [2698, 40], [2699, 58], [2700, 46], [2701, 26], [2702, 46], [2703, 52], [2704, 46], [2705, 76], [2706, 74], [2708, 128], [2709, 78], [2710, 26], [2741, 20], [2742, 58], [2743, 26], [2745, 72], [2746, 22], [2747, 26], [2748, 60], [2749, 136], [2750, 200], [4066, 22], [4067, 14], [4068, 73], [4069, 20], [4073, 26], [4310, 26], [4311, 14], [4312, 26], [4313, 46], [4314, 32], [4630, 20], [4631, 14], [4632, 14], [4633, 14], [4634, 18], [4635, 34], [4641, 26], [4642, 40], [4643, 24], [4674, 16], [4675, 48], [4676, 34], [4840, 8]]
learning.log: DEBUG: TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4559
learning.log: DEBUG: TextArray#__init__ start
learning.log: DEBUG: TextArray#to_vec start
learning.log: DEBUG: TextArray#to_vec end
learning.log: DEBUG: use algorithm: Multi-layer Perceptron
learning.log: DEBUG: learning_parameter has not parameter [activation]
learning.log: DEBUG: best_params_: {'activation': 'identity'}
learning.log: DEBUG: after Bot#__get_estimator
learning.log: DEBUG: before Evaluator#evaluate
learning.log: DEBUG: self.threshold: 0.0
learning.log: DEBUG: Evaluator#evaluate#elapsed time: 6253.784895 ms
learning.log: DEBUG: accuracy: 0.97458369851
learning.log: DEBUG: end Bot#learn
learning.log: DEBUG: Bot.learn processing time: 109.80081009864807 [sec]
learning.log: DEBUG: TextArray#__init__ start
learning.log: DEBUG: Reply#predict text_array.separated_sentences: ['']
learning.log: DEBUG: TextArray#to_vec start
learning.log: DEBUG: TextArray#to_vec end
learning.log: DEBUG: Reply#predict features: 
--------------------- >> end captured logging << ---------------------

======================================================================
FAIL: test_fail_want_to_eat_ramen (learning.tests.engine.test_ptna_conversation.PtnaConversationTestCase)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/Users/makmorit/GitHub/donusagi-bot/learning/learning/tests/engine/test_ptna_conversation.py", line 58, in test_fail_want_to_eat_ramen
    ok_(result.probability > self.threshold)
AssertionError: None
-------------------- >> begin captured logging << --------------------
learning.log: DEBUG: learning_parameter: {'_include_failed_data': False, '_params_for_algorithm': {}, '_include_tag_vector': False, '_algorithm': 2, '_classify_threshold': None}
learning.log: DEBUG: start Bot#learn
learning.log: DEBUG: TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4559
learning.log: DEBUG: TextArray#__init__ start
learning.log: DEBUG: TextArray#to_vec start
learning.log: DEBUG: TextArray#to_vec end
learning.log: DEBUG: Bot#__build_training_set training_set.count_sample_by_y: [[0, 3], [2671, 226], [2673, 166], [2674, 32], [2675, 152], [2676, 130], [2677, 124], [2678, 72], [2679, 150], [2680, 78], [2681, 96], [2682, 66], [2683, 96], [2684, 46], [2685, 40], [2686, 52], [2687, 142], [2688, 142], [2689, 176], [2690, 66], [2691, 46], [2692, 156], [2694, 76], [2695, 72], [2696, 90], [2697, 142], [2698, 40], [2699, 58], [2700, 46], [2701, 26], [2702, 46], [2703, 52], [2704, 46], [2705, 76], [2706, 74], [2708, 128], [2709, 78], [2710, 26], [2741, 20], [2742, 58], [2743, 26], [2745, 72], [2746, 22], [2747, 26], [2748, 60], [2749, 136], [2750, 200], [4066, 22], [4067, 14], [4068, 73], [4069, 20], [4073, 26], [4310, 26], [4311, 14], [4312, 26], [4313, 46], [4314, 32], [4630, 20], [4631, 14], [4632, 14], [4633, 14], [4634, 18], [4635, 34], [4641, 26], [4642, 40], [4643, 24], [4674, 16], [4675, 48], [4676, 34], [4840, 8]]
learning.log: DEBUG: TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4559
learning.log: DEBUG: TextArray#__init__ start
learning.log: DEBUG: TextArray#to_vec start
learning.log: DEBUG: TextArray#to_vec end
learning.log: DEBUG: use algorithm: Multi-layer Perceptron
learning.log: DEBUG: learning_parameter has not parameter [activation]
learning.log: DEBUG: best_params_: {'activation': 'identity'}
learning.log: DEBUG: after Bot#__get_estimator
learning.log: DEBUG: before Evaluator#evaluate
learning.log: DEBUG: self.threshold: 0.0
learning.log: DEBUG: Evaluator#evaluate#elapsed time: 6280.805826 ms
learning.log: DEBUG: accuracy: 0.97458369851
learning.log: DEBUG: end Bot#learn
learning.log: DEBUG: Bot.learn processing time: 109.72946405410767 [sec]
learning.log: DEBUG: TextArray#__init__ start
learning.log: DEBUG: Reply#predict text_array.separated_sentences: ['おいしい ラーメン 食べる']
learning.log: DEBUG: TextArray#to_vec start
learning.log: DEBUG: TextArray#to_vec end
learning.log: DEBUG: Reply#predict features: 
--------------------- >> end captured logging << ---------------------

----------------------------------------------------------------------
Ran 22 tests in 1849.570s

FAILED (failures=2)
MacBookPro-makmorit-jp:learning makmorit$ date
Tue Apr 11 15:28:02 JST 2017
MacBookPro-makmorit-jp:learning makmorit$ 
```

## (3) テスト時のログ

ご参考までに、全量掲載させていただきます。

```
2017-04-11 14:55:35,184 learning_parameter: {'_include_failed_data': False, '_params_for_algorithm': {}, '_include_tag_vector': False, '_algorithm': 2, '_classify_threshold': None}
2017-04-11 14:55:35,184 start Bot#learn
2017-04-11 14:55:35,186 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 6
2017-04-11 14:55:35,186 TextArray#__init__ start
2017-04-11 14:55:35,190 TextArray#to_vec start
2017-04-11 14:55:35,191 TextArray#to_vec end
2017-04-11 14:55:35,191 Bot#__build_training_set training_set.count_sample_by_y: [[0, 3], [1, 4], [2, 2]]
2017-04-11 14:55:35,192 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 6
2017-04-11 14:55:35,192 TextArray#__init__ start
2017-04-11 14:55:35,196 TextArray#to_vec start
2017-04-11 14:55:35,197 TextArray#to_vec end
2017-04-11 14:55:35,197 use algorithm: Multi-layer Perceptron
2017-04-11 14:55:35,197 learning_parameter has not parameter [activation]
2017-04-11 14:55:39,132 best_params_: {'activation': 'identity'}
2017-04-11 14:55:39,132 after Bot#__get_estimator
2017-04-11 14:55:39,149 before Evaluator#evaluate
2017-04-11 14:55:39,149 self.threshold: 0.0
2017-04-11 14:55:39,370 Evaluator#evaluate#elapsed time: 220.394135 ms
2017-04-11 14:55:39,370 accuracy: 1.0
2017-04-11 14:55:39,370 end Bot#learn
2017-04-11 14:55:39,370 Bot.learn processing time: 4.1862359046936035 [sec]
2017-04-11 14:55:39,385 TextArray#__init__ start
2017-04-11 14:55:39,386 Reply#predict text_array.separated_sentences: ['Windows ログイン 出来る ない']
2017-04-11 14:55:39,386 TextArray#to_vec start
2017-04-11 14:55:39,386 TextArray#to_vec end
2017-04-11 14:55:39,386 Reply#predict features:   (0, 1)	0.5
  (0, 3)	0.5
  (0, 11)	0.5
  (0, 12)	0.5
2017-04-11 14:55:39,387 TextArray#__init__ start
2017-04-11 14:55:39,388 TextArray#to_vec start
2017-04-11 14:55:39,389 TextArray#to_vec end
2017-04-11 14:55:39,407 learning_parameter: {'_include_failed_data': False, '_params_for_algorithm': {}, '_include_tag_vector': False, '_algorithm': 2, '_classify_threshold': None}
2017-04-11 14:55:39,407 start Bot#learn
2017-04-11 14:55:39,418 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4114
2017-04-11 14:55:39,418 TextArray#__init__ start
2017-04-11 14:55:40,895 TextArray#to_vec start
2017-04-11 14:55:40,937 TextArray#to_vec end
2017-04-11 14:55:40,938 Bot#__build_training_set training_set.count_sample_by_y: [[0, 3], [4677, 14], [4678, 2584], [4679, 14], [4680, 14], [4683, 14], [4686, 14], [4687, 8], [4690, 8], [4691, 14], [4692, 44], [4693, 30], [4700, 12], [4707, 18], [4708, 14], [4709, 14], [4710, 14], [4711, 14], [4712, 16], [4713, 14], [4718, 20], [4719, 8], [4720, 20], [4721, 8], [4724, 8], [4727, 14], [4728, 62], [4729, 26], [4730, 20], [4731, 14], [4732, 22], [4733, 14], [4734, 34], [4735, 12], [4738, 14], [4739, 14], [4740, 14], [4741, 8], [4742, 20], [4743, 14], [4744, 14], [4745, 20], [4750, 22], [4751, 20], [4752, 14], [4753, 8], [4754, 8], [4755, 14], [4756, 20], [4757, 8], [4758, 28], [4759, 14], [4760, 8], [4761, 20], [4762, 8], [4766, 14], [4767, 14], [4772, 60], [4776, 8], [4781, 14], [4782, 14], [4783, 20], [4786, 8], [4792, 20], [4793, 8], [4794, 8], [4795, 14], [4797, 42], [4798, 36], [4800, 14], [4802, 14], [4804, 14], [4807, 8], [4808, 106], [4817, 26], [4821, 24], [4824, 16], [4827, 8], [4830, 22], [4831, 14], [4833, 14], [4834, 14], [4835, 36], [4836, 30], [4837, 8]]
2017-04-11 14:55:40,947 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4114
2017-04-11 14:55:40,947 TextArray#__init__ start
2017-04-11 14:55:42,413 TextArray#to_vec start
2017-04-11 14:55:42,456 TextArray#to_vec end
2017-04-11 14:55:42,456 use algorithm: Multi-layer Perceptron
2017-04-11 14:55:42,456 learning_parameter has not parameter [activation]
2017-04-11 14:57:24,906 best_params_: {'activation': 'relu'}
2017-04-11 14:57:24,906 after Bot#__get_estimator
2017-04-11 14:57:24,942 before Evaluator#evaluate
2017-04-11 14:57:24,942 self.threshold: 0.0
2017-04-11 14:57:30,734 Evaluator#evaluate#elapsed time: 5792.215109 ms
2017-04-11 14:57:30,734 accuracy: 0.983495145631
2017-04-11 14:57:30,734 end Bot#learn
2017-04-11 14:57:30,735 Bot.learn processing time: 111.32778286933899 [sec]
2017-04-11 14:57:30,757 TextArray#__init__ start
2017-04-11 14:57:30,758 Reply#predict text_array.separated_sentences: ['EX カード 貸す']
2017-04-11 14:57:30,758 TextArray#to_vec start
2017-04-11 14:57:30,759 TextArray#to_vec end
2017-04-11 14:57:30,759 Reply#predict features:   (0, 7)	0.57735026919
  (0, 124)	0.57735026919
  (0, 593)	0.57735026919
2017-04-11 14:57:30,789 TextArray#__init__ start
2017-04-11 14:57:30,790 Reply#predict text_array.separated_sentences: ['契約 書 見る']
2017-04-11 14:57:30,790 TextArray#to_vec start
2017-04-11 14:57:30,790 TextArray#to_vec end
2017-04-11 14:57:30,790 Reply#predict features:   (0, 375)	0.57735026919
  (0, 452)	0.57735026919
  (0, 570)	0.57735026919
2017-04-11 14:57:30,858 learning_parameter: {'_include_failed_data': False, '_params_for_algorithm': {'C': 200}, '_include_tag_vector': False, '_algorithm': 2, '_classify_threshold': None}
2017-04-11 14:57:30,858 start Bot#learn
2017-04-11 14:57:30,919 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17443
2017-04-11 14:57:30,919 TextArray#__init__ start
2017-04-11 14:57:40,283 TextArray#to_vec start
2017-04-11 14:57:40,677 TextArray#to_vec end
2017-04-11 14:57:40,680 Bot#__build_training_set training_set.count_sample_by_y: [[0, 3], [3397, 24], [3398, 54], [3399, 30], [3400, 26], [3401, 42], [3402, 42], [3404, 44], [3405, 42], [3406, 36], [3407, 36], [3408, 36], [3409, 42], [3411, 26], [3412, 30], [3413, 36], [3414, 32], [3415, 38], [3416, 32], [3417, 32], [3418, 30], [3419, 42], [3420, 32], [3421, 36], [3422, 46], [3423, 38], [3424, 42], [3425, 36], [3426, 38], [3427, 32], [3428, 36], [3429, 38], [3430, 30], [3431, 36], [3432, 38], [3433, 54], [3434, 38], [3435, 32], [3436, 52], [3437, 30], [3438, 38], [3439, 38], [3440, 26], [3441, 30], [3442, 30], [3443, 32], [3444, 32], [3445, 36], [3446, 36], [3447, 32], [3449, 30], [3451, 36], [3452, 38], [3453, 26], [3454, 42], [3455, 42], [3456, 42], [3457, 36], [3458, 36], [3459, 30], [3460, 42], [3461, 48], [3462, 36], [3463, 26], [3464, 36], [3465, 38], [3466, 24], [3467, 36], [3468, 32], [3469, 36], [3470, 36], [3471, 32], [3472, 30], [3473, 32], [3474, 38], [3475, 36], [3476, 20], [3477, 66], [3478, 38], [3479, 38], [3480, 40], [3481, 26], [3482, 42], [3483, 42], [3484, 42], [3485, 46], [3486, 42], [3487, 42], [3488, 48], [3489, 26], [3490, 32], [3491, 36], [3492, 32], [3493, 36], [3494, 38], [3496, 36], [3497, 49], [3498, 24], [3499, 20], [3500, 38], [3501, 28], [3502, 34], [3503, 38], [3504, 32], [3505, 34], [3506, 32], [3507, 32], [3508, 38], [3509, 26], [3510, 26], [3511, 38], [3512, 32], [3513, 32], [3514, 32], [3515, 28], [3516, 32], [3517, 38], [3518, 82], [3519, 32], [3520, 32], [3521, 26], [3522, 36], [3523, 36], [3524, 26], [3525, 26], [3526, 32], [3527, 62], [3528, 32], [3529, 40], [3530, 32], [3531, 34], [3532, 32], [3534, 32], [3535, 38], [3536, 26], [3537, 26], [3538, 38], [3539, 32], [3540, 32], [3541, 32], [3543, 32], [3544, 38], [3545, 32], [3546, 38], [3547, 38], [3548, 34], [3549, 32], [3550, 26], [3551, 44], [3552, 32], [3553, 32], [3554, 32], [3555, 32], [3556, 26], [3557, 34], [3558, 38], [3559, 26], [3560, 38], [3561, 32], [3562, 32], [3563, 32], [3564, 28], [3565, 38], [3567, 34], [3568, 38], [3569, 32], [3570, 32], [3571, 32], [3572, 38], [3573, 38], [3574, 22], [3575, 26], [3576, 32], [3577, 32], [3578, 38], [3579, 26], [3580, 26], [3581, 32], [3582, 32], [3583, 38], [3584, 26], [3585, 38], [3586, 38], [3587, 38], [3588, 46], [3589, 32], [3590, 34], [3591, 40], [3592, 32], [3593, 32], [3594, 26], [3595, 32], [3596, 56], [3597, 38], [3598, 38], [3599, 26], [3600, 38], [3601, 32], [3602, 26], [3603, 32], [3604, 26], [3605, 32], [3606, 40], [3607, 32], [3608, 32], [3609, 32], [3610, 28], [3611, 32], [3612, 32], [3613, 32], [3615, 38], [3616, 26], [3617, 32], [3618, 38], [3619, 38], [3620, 32], [3621, 28], [3622, 44], [3623, 32], [3624, 38], [3625, 34], [3626, 32], [3627, 38], [3628, 34], [3629, 44], [3630, 26], [3631, 32], [3632, 32], [3633, 44], [3634, 32], [3635, 40], [3636, 38], [3637, 40], [3638, 34], [3639, 32], [3640, 38], [3641, 32], [3642, 34], [3643, 42], [3644, 32], [3645, 36], [3646, 32], [3647, 32], [3648, 32], [3649, 32], [3650, 32], [3651, 38], [3652, 32], [3653, 38], [3654, 32], [3655, 32], [3656, 32], [3657, 26], [3658, 32], [3659, 32], [3660, 32], [3661, 26], [3662, 32], [3663, 62], [3664, 38], [3665, 32], [3666, 32], [3667, 38], [3668, 32], [3669, 38], [3670, 38], [3671, 26], [3672, 40], [3673, 38], [3674, 32], [3675, 38], [3676, 38], [3677, 42], [3678, 32], [3679, 38], [3680, 34], [3681, 32], [3682, 32], [3683, 70], [3684, 98], [3685, 38], [3686, 32], [3687, 32], [3688, 32], [3689, 38], [3690, 38], [3691, 26], [3692, 44], [3693, 38], [3694, 26], [3695, 44], [3697, 42], [3698, 36], [3699, 30], [3700, 36], [3701, 50], [3702, 36], [3703, 42], [3704, 38], [3705, 42], [3706, 42], [3707, 36], [3708, 30], [3709, 42], [3710, 36], [3711, 36], [3712, 36], [3713, 36], [3714, 44], [3715, 36], [3716, 44], [3717, 36], [3718, 54], [3719, 42], [3720, 36], [3721, 58], [3722, 38], [3723, 42], [3724, 42], [3725, 38], [3726, 38], [3727, 36], [3728, 24], [3729, 42], [3730, 48], [3731, 46], [3732, 42], [3733, 36], [3734, 42], [3736, 40], [3737, 38], [3738, 38], [3739, 32], [3740, 40], [3741, 32], [3753, 8], [3754, 44], [3755, 24], [3756, 24], [3757, 30], [3758, 24], [3759, 30], [3760, 24], [3761, 30], [3762, 32], [3763, 40], [3764, 24], [3765, 30], [3766, 24], [3767, 62], [3768, 20], [3769, 26], [3770, 20], [3771, 26], [3772, 26], [3773, 26], [3774, 24], [3775, 34], [3776, 22], [3777, 26], [3778, 14], [3779, 26], [3780, 20], [3781, 14], [3782, 26], [3783, 22], [3784, 20], [3785, 34], [3786, 20], [3787, 28], [3788, 20], [3789, 26], [3790, 20], [3791, 30], [3792, 20], [3793, 14], [3794, 24], [3795, 34], [3796, 26], [3797, 44], [3798, 32], [3799, 28], [3801, 38], [3802, 26], [3803, 20], [3804, 26], [3805, 28], [3806, 20], [3807, 20], [3808, 26], [3809, 52], [3810, 16], [3812, 26], [3813, 32], [3814, 26], [3815, 34], [3816, 38], [3817, 22], [3818, 30], [3819, 28], [3820, 24], [3821, 32], [3822, 14], [3823, 26], [3824, 34], [3825, 18], [3826, 14], [3827, 20], [3828, 26], [3829, 20], [3830, 32], [3831, 26], [3832, 32], [3833, 14], [3834, 20], [3835, 30], [3836, 24], [3837, 20], [3838, 32], [3839, 22], [3840, 22], [3841, 26], [3842, 32], [3843, 26], [3844, 32], [3845, 26], [3846, 26], [3848, 28], [3849, 20], [3850, 20], [3851, 26], [3852, 20], [3853, 14], [3854, 20], [3855, 20], [3856, 26], [3857, 26], [3858, 28], [3859, 8], [3860, 38], [3861, 28], [3862, 32], [3864, 20], [3865, 20], [3866, 26], [3867, 26], [3868, 20], [3869, 26], [3870, 24], [3871, 14], [3872, 24], [3873, 36], [3874, 26], [3875, 36], [3876, 26], [3877, 32], [3878, 20], [3879, 26], [3880, 26], [3881, 20], [3882, 26], [3883, 20], [3884, 30], [3885, 26], [3886, 14], [3887, 26], [3888, 20], [3889, 20], [3890, 20], [3891, 26], [3892, 26], [3893, 32], [3894, 26], [3895, 20], [3896, 20], [3897, 20], [3898, 32], [3899, 32], [3900, 30], [3901, 26], [3904, 26], [3905, 26], [3906, 32], [3907, 26], [3908, 14], [3909, 24], [3910, 38], [3911, 20], [3912, 24], [3913, 32], [3914, 26], [3915, 36], [3916, 20], [3917, 56], [3918, 24], [3919, 24], [3920, 24], [3931, 2], [3933, 2], [3934, 8], [3935, 2], [3936, 8], [3938, 2], [3939, 42], [3940, 42], [3941, 38], [3942, 30], [3943, 36], [3944, 38], [3945, 26], [3946, 38], [3947, 38], [3948, 18], [3949, 36], [3950, 20], [3951, 26], [3952, 32], [3953, 36], [3954, 26], [3955, 26], [3956, 4], [3957, 20], [3958, 8], [3959, 32], [3960, 20], [3961, 30], [3962, 30], [3963, 6], [3964, 46], [3965, 20], [3966, 6], [3967, 6], [3968, 12], [3969, 6], [3970, 6], [3971, 6], [3972, 8], [3973, 2], [3974, 6], [3975, 16], [3976, 2], [3977, 2], [3978, 12], [3979, 2], [3980, 8], [3981, 2], [3982, 14], [3983, 8], [3984, 2], [3985, 10], [3986, 14], [3987, 8], [3988, 2], [3989, 2], [3990, 8], [3991, 2], [3992, 14], [3993, 2], [3994, 2], [3995, 2], [3996, 2], [3997, 2], [3998, 2], [3999, 14], [4000, 2], [4001, 8], [4002, 4], [4003, 2], [4004, 10], [4005, 2], [4006, 14], [4007, 8], [4008, 8], [4009, 24], [4010, 12], [4011, 20], [4012, 16], [4013, 20], [4014, 14], [4015, 8], [4016, 4], [4017, 22], [4018, 16], [4019, 4], [4020, 10], [4021, 10], [4644, 2], [4645, 2], [4646, 2], [4647, 2], [4648, 6], [4649, 2], [4650, 2], [4651, 2], [4652, 2], [4653, 2], [4654, 2], [4655, 2], [4656, 2], [4657, 6], [4658, 6], [4659, 2], [4660, 2], [4661, 2], [4663, 2], [4664, 6], [4665, 6], [4666, 2], [4667, 6], [4668, 2], [4669, 2], [4670, 2], [4671, 6], [4672, 2]]
2017-04-11 14:57:40,747 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 17443
2017-04-11 14:57:40,747 TextArray#__init__ start
2017-04-11 14:57:50,093 TextArray#to_vec start
2017-04-11 14:57:50,470 TextArray#to_vec end
2017-04-11 14:57:50,472 use algorithm: Multi-layer Perceptron
2017-04-11 14:57:50,472 learning_parameter has not parameter [activation]
2017-04-11 15:16:03,753 best_params_: {'activation': 'identity'}
2017-04-11 15:16:03,753 after Bot#__get_estimator
2017-04-11 15:16:03,861 before Evaluator#evaluate
2017-04-11 15:16:03,861 self.threshold: 0.0
2017-04-11 15:17:15,720 Evaluator#evaluate#elapsed time: 71859.011173 ms
2017-04-11 15:17:15,721 accuracy: 0.986474094452
2017-04-11 15:17:15,721 end Bot#learn
2017-04-11 15:17:15,729 Bot.learn processing time: 1184.8716039657593 [sec]
2017-04-11 15:17:15,776 TextArray#__init__ start
2017-04-11 15:17:15,777 Reply#predict text_array.separated_sentences: ['しいたけ 嫌い 食べ物']
2017-04-11 15:17:15,777 TextArray#to_vec start
2017-04-11 15:17:15,777 TextArray#to_vec end
2017-04-11 15:17:15,777 Reply#predict features: 
2017-04-11 15:17:15,822 TextArray#__init__ start
2017-04-11 15:17:15,823 Reply#predict text_array.separated_sentences: ['社外 メール 添付 ファイル 受信 容量 制限']
2017-04-11 15:17:15,823 TextArray#to_vec start
2017-04-11 15:17:15,824 TextArray#to_vec end
2017-04-11 15:17:15,824 Reply#predict features:   (0, 716)	0.377964473009
  (0, 785)	0.377964473009
  (0, 1058)	0.377964473009
  (0, 1110)	0.377964473009
  (0, 1222)	0.377964473009
  (0, 1516)	0.377964473009
  (0, 1598)	0.377964473009
2017-04-11 15:17:15,869 TextArray#__init__ start
2017-04-11 15:17:15,870 Reply#predict text_array.separated_sentences: ['メール 署名 付ける 方法 知る']
2017-04-11 15:17:15,870 TextArray#to_vec start
2017-04-11 15:17:15,870 TextArray#to_vec end
2017-04-11 15:17:15,870 Reply#predict features:   (0, 785)	0.4472135955
  (0, 919)	0.4472135955
  (0, 1381)	0.4472135955
  (0, 1586)	0.4472135955
  (0, 1654)	0.4472135955
2017-04-11 15:17:15,916 TextArray#__init__ start
2017-04-11 15:17:15,917 Reply#predict text_array.separated_sentences: ['SPAM フォルダ メール 入る']
2017-04-11 15:17:15,917 TextArray#to_vec start
2017-04-11 15:17:15,917 TextArray#to_vec end
2017-04-11 15:17:15,917 Reply#predict features:   (0, 223)	0.5
  (0, 723)	0.5
  (0, 785)	0.5
  (0, 991)	0.5
2017-04-11 15:17:15,934 learning_parameter: {'_include_failed_data': False, '_params_for_algorithm': {}, '_include_tag_vector': False, '_algorithm': 2, '_classify_threshold': None}
2017-04-11 15:17:15,934 start Bot#learn
2017-04-11 15:17:15,948 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4559
2017-04-11 15:17:15,949 TextArray#__init__ start
2017-04-11 15:17:17,792 TextArray#to_vec start
2017-04-11 15:17:17,847 TextArray#to_vec end
2017-04-11 15:17:17,848 Bot#__build_training_set training_set.count_sample_by_y: [[0, 3], [2671, 226], [2673, 166], [2674, 32], [2675, 152], [2676, 130], [2677, 124], [2678, 72], [2679, 150], [2680, 78], [2681, 96], [2682, 66], [2683, 96], [2684, 46], [2685, 40], [2686, 52], [2687, 142], [2688, 142], [2689, 176], [2690, 66], [2691, 46], [2692, 156], [2694, 76], [2695, 72], [2696, 90], [2697, 142], [2698, 40], [2699, 58], [2700, 46], [2701, 26], [2702, 46], [2703, 52], [2704, 46], [2705, 76], [2706, 74], [2708, 128], [2709, 78], [2710, 26], [2741, 20], [2742, 58], [2743, 26], [2745, 72], [2746, 22], [2747, 26], [2748, 60], [2749, 136], [2750, 200], [4066, 22], [4067, 14], [4068, 73], [4069, 20], [4073, 26], [4310, 26], [4311, 14], [4312, 26], [4313, 46], [4314, 32], [4630, 20], [4631, 14], [4632, 14], [4633, 14], [4634, 18], [4635, 34], [4641, 26], [4642, 40], [4643, 24], [4674, 16], [4675, 48], [4676, 34], [4840, 8]]
2017-04-11 15:17:17,861 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4559
2017-04-11 15:17:17,861 TextArray#__init__ start
2017-04-11 15:17:19,678 TextArray#to_vec start
2017-04-11 15:17:19,732 TextArray#to_vec end
2017-04-11 15:17:19,733 use algorithm: Multi-layer Perceptron
2017-04-11 15:17:19,733 learning_parameter has not parameter [activation]
2017-04-11 15:18:59,443 best_params_: {'activation': 'identity'}
2017-04-11 15:18:59,443 after Bot#__get_estimator
2017-04-11 15:18:59,479 before Evaluator#evaluate
2017-04-11 15:18:59,480 self.threshold: 0.0
2017-04-11 15:19:05,734 Evaluator#evaluate#elapsed time: 6253.784895 ms
2017-04-11 15:19:05,734 accuracy: 0.97458369851
2017-04-11 15:19:05,734 end Bot#learn
2017-04-11 15:19:05,735 Bot.learn processing time: 109.80081009864807 [sec]
2017-04-11 15:19:05,758 TextArray#__init__ start
2017-04-11 15:19:05,759 Reply#predict text_array.separated_sentences: ['']
2017-04-11 15:19:05,759 TextArray#to_vec start
2017-04-11 15:19:05,759 TextArray#to_vec end
2017-04-11 15:19:05,759 Reply#predict features: 
2017-04-11 15:19:05,777 learning_parameter: {'_include_failed_data': False, '_params_for_algorithm': {}, '_include_tag_vector': False, '_algorithm': 2, '_classify_threshold': None}
2017-04-11 15:19:05,777 start Bot#learn
2017-04-11 15:19:05,787 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4559
2017-04-11 15:19:05,788 TextArray#__init__ start
2017-04-11 15:19:07,566 TextArray#to_vec start
2017-04-11 15:19:07,621 TextArray#to_vec end
2017-04-11 15:19:07,622 Bot#__build_training_set training_set.count_sample_by_y: [[0, 3], [2671, 226], [2673, 166], [2674, 32], [2675, 152], [2676, 130], [2677, 124], [2678, 72], [2679, 150], [2680, 78], [2681, 96], [2682, 66], [2683, 96], [2684, 46], [2685, 40], [2686, 52], [2687, 142], [2688, 142], [2689, 176], [2690, 66], [2691, 46], [2692, 156], [2694, 76], [2695, 72], [2696, 90], [2697, 142], [2698, 40], [2699, 58], [2700, 46], [2701, 26], [2702, 46], [2703, 52], [2704, 46], [2705, 76], [2706, 74], [2708, 128], [2709, 78], [2710, 26], [2741, 20], [2742, 58], [2743, 26], [2745, 72], [2746, 22], [2747, 26], [2748, 60], [2749, 136], [2750, 200], [4066, 22], [4067, 14], [4068, 73], [4069, 20], [4073, 26], [4310, 26], [4311, 14], [4312, 26], [4313, 46], [4314, 32], [4630, 20], [4631, 14], [4632, 14], [4633, 14], [4634, 18], [4635, 34], [4641, 26], [4642, 40], [4643, 24], [4674, 16], [4675, 48], [4676, 34], [4840, 8]]
2017-04-11 15:19:07,634 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4559
2017-04-11 15:19:07,634 TextArray#__init__ start
2017-04-11 15:19:09,410 TextArray#to_vec start
2017-04-11 15:19:09,462 TextArray#to_vec end
2017-04-11 15:19:09,462 use algorithm: Multi-layer Perceptron
2017-04-11 15:19:09,462 learning_parameter has not parameter [activation]
2017-04-11 15:20:49,189 best_params_: {'activation': 'identity'}
2017-04-11 15:20:49,190 after Bot#__get_estimator
2017-04-11 15:20:49,224 before Evaluator#evaluate
2017-04-11 15:20:49,224 self.threshold: 0.0
2017-04-11 15:20:55,505 Evaluator#evaluate#elapsed time: 6280.805826 ms
2017-04-11 15:20:55,505 accuracy: 0.97458369851
2017-04-11 15:20:55,506 end Bot#learn
2017-04-11 15:20:55,506 Bot.learn processing time: 109.72946405410767 [sec]
2017-04-11 15:20:55,530 TextArray#__init__ start
2017-04-11 15:20:55,530 Reply#predict text_array.separated_sentences: ['おいしい ラーメン 食べる']
2017-04-11 15:20:55,530 TextArray#to_vec start
2017-04-11 15:20:55,531 TextArray#to_vec end
2017-04-11 15:20:55,531 Reply#predict features: 
2017-04-11 15:20:55,546 learning_parameter: {'_include_failed_data': False, '_params_for_algorithm': {}, '_include_tag_vector': False, '_algorithm': 2, '_classify_threshold': None}
2017-04-11 15:20:55,547 start Bot#learn
2017-04-11 15:20:55,558 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4559
2017-04-11 15:20:55,558 TextArray#__init__ start
2017-04-11 15:20:57,292 TextArray#to_vec start
2017-04-11 15:20:57,343 TextArray#to_vec end
2017-04-11 15:20:57,343 Bot#__build_training_set training_set.count_sample_by_y: [[0, 3], [2671, 226], [2673, 166], [2674, 32], [2675, 152], [2676, 130], [2677, 124], [2678, 72], [2679, 150], [2680, 78], [2681, 96], [2682, 66], [2683, 96], [2684, 46], [2685, 40], [2686, 52], [2687, 142], [2688, 142], [2689, 176], [2690, 66], [2691, 46], [2692, 156], [2694, 76], [2695, 72], [2696, 90], [2697, 142], [2698, 40], [2699, 58], [2700, 46], [2701, 26], [2702, 46], [2703, 52], [2704, 46], [2705, 76], [2706, 74], [2708, 128], [2709, 78], [2710, 26], [2741, 20], [2742, 58], [2743, 26], [2745, 72], [2746, 22], [2747, 26], [2748, 60], [2749, 136], [2750, 200], [4066, 22], [4067, 14], [4068, 73], [4069, 20], [4073, 26], [4310, 26], [4311, 14], [4312, 26], [4313, 46], [4314, 32], [4630, 20], [4631, 14], [4632, 14], [4633, 14], [4634, 18], [4635, 34], [4641, 26], [4642, 40], [4643, 24], [4674, 16], [4675, 48], [4676, 34], [4840, 8]]
2017-04-11 15:20:57,355 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4559
2017-04-11 15:20:57,355 TextArray#__init__ start
2017-04-11 15:20:59,155 TextArray#to_vec start
2017-04-11 15:20:59,221 TextArray#to_vec end
2017-04-11 15:20:59,221 use algorithm: Multi-layer Perceptron
2017-04-11 15:20:59,221 learning_parameter has not parameter [activation]
2017-04-11 15:22:38,303 best_params_: {'activation': 'identity'}
2017-04-11 15:22:38,304 after Bot#__get_estimator
2017-04-11 15:22:38,338 before Evaluator#evaluate
2017-04-11 15:22:38,339 self.threshold: 0.0
2017-04-11 15:22:44,660 Evaluator#evaluate#elapsed time: 6320.868015 ms
2017-04-11 15:22:44,660 accuracy: 0.97458369851
2017-04-11 15:22:44,660 end Bot#learn
2017-04-11 15:22:44,660 Bot.learn processing time: 109.11351084709167 [sec]
2017-04-11 15:22:44,684 TextArray#__init__ start
2017-04-11 15:22:44,684 Reply#predict text_array.separated_sentences: ['こんにちは']
2017-04-11 15:22:44,684 TextArray#to_vec start
2017-04-11 15:22:44,685 TextArray#to_vec end
2017-04-11 15:22:44,685 Reply#predict features:   (0, 74)	1.0
2017-04-11 15:22:44,701 learning_parameter: {'_include_failed_data': False, '_params_for_algorithm': {}, '_include_tag_vector': False, '_algorithm': 2, '_classify_threshold': None}
2017-04-11 15:22:44,701 start Bot#learn
2017-04-11 15:22:44,716 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4559
2017-04-11 15:22:44,717 TextArray#__init__ start
2017-04-11 15:22:46,515 TextArray#to_vec start
2017-04-11 15:22:46,567 TextArray#to_vec end
2017-04-11 15:22:46,567 Bot#__build_training_set training_set.count_sample_by_y: [[0, 3], [2671, 226], [2673, 166], [2674, 32], [2675, 152], [2676, 130], [2677, 124], [2678, 72], [2679, 150], [2680, 78], [2681, 96], [2682, 66], [2683, 96], [2684, 46], [2685, 40], [2686, 52], [2687, 142], [2688, 142], [2689, 176], [2690, 66], [2691, 46], [2692, 156], [2694, 76], [2695, 72], [2696, 90], [2697, 142], [2698, 40], [2699, 58], [2700, 46], [2701, 26], [2702, 46], [2703, 52], [2704, 46], [2705, 76], [2706, 74], [2708, 128], [2709, 78], [2710, 26], [2741, 20], [2742, 58], [2743, 26], [2745, 72], [2746, 22], [2747, 26], [2748, 60], [2749, 136], [2750, 200], [4066, 22], [4067, 14], [4068, 73], [4069, 20], [4073, 26], [4310, 26], [4311, 14], [4312, 26], [4313, 46], [4314, 32], [4630, 20], [4631, 14], [4632, 14], [4633, 14], [4634, 18], [4635, 34], [4641, 26], [4642, 40], [4643, 24], [4674, 16], [4675, 48], [4676, 34], [4840, 8]]
2017-04-11 15:22:46,579 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4559
2017-04-11 15:22:46,580 TextArray#__init__ start
2017-04-11 15:22:48,324 TextArray#to_vec start
2017-04-11 15:22:48,377 TextArray#to_vec end
2017-04-11 15:22:48,377 use algorithm: Multi-layer Perceptron
2017-04-11 15:22:48,377 learning_parameter has not parameter [activation]
2017-04-11 15:24:28,696 best_params_: {'activation': 'identity'}
2017-04-11 15:24:28,696 after Bot#__get_estimator
2017-04-11 15:24:28,731 before Evaluator#evaluate
2017-04-11 15:24:28,732 self.threshold: 0.0
2017-04-11 15:24:35,031 Evaluator#evaluate#elapsed time: 6298.924923 ms
2017-04-11 15:24:35,031 accuracy: 0.97458369851
2017-04-11 15:24:35,031 end Bot#learn
2017-04-11 15:24:35,032 Bot.learn processing time: 110.33035778999329 [sec]
2017-04-11 15:24:35,056 TextArray#__init__ start
2017-04-11 15:24:35,057 Reply#predict text_array.separated_sentences: ['女 先生']
2017-04-11 15:24:35,057 TextArray#to_vec start
2017-04-11 15:24:35,057 TextArray#to_vec end
2017-04-11 15:24:35,057 Reply#predict features:   (0, 262)	0.707106781187
  (0, 352)	0.707106781187
2017-04-11 15:24:35,076 learning_parameter: {'_include_failed_data': False, '_params_for_algorithm': {}, '_include_tag_vector': False, '_algorithm': 2, '_classify_threshold': None}
2017-04-11 15:24:35,076 start Bot#learn
2017-04-11 15:24:35,088 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4559
2017-04-11 15:24:35,088 TextArray#__init__ start
2017-04-11 15:24:36,848 TextArray#to_vec start
2017-04-11 15:24:36,898 TextArray#to_vec end
2017-04-11 15:24:36,899 Bot#__build_training_set training_set.count_sample_by_y: [[0, 3], [2671, 226], [2673, 166], [2674, 32], [2675, 152], [2676, 130], [2677, 124], [2678, 72], [2679, 150], [2680, 78], [2681, 96], [2682, 66], [2683, 96], [2684, 46], [2685, 40], [2686, 52], [2687, 142], [2688, 142], [2689, 176], [2690, 66], [2691, 46], [2692, 156], [2694, 76], [2695, 72], [2696, 90], [2697, 142], [2698, 40], [2699, 58], [2700, 46], [2701, 26], [2702, 46], [2703, 52], [2704, 46], [2705, 76], [2706, 74], [2708, 128], [2709, 78], [2710, 26], [2741, 20], [2742, 58], [2743, 26], [2745, 72], [2746, 22], [2747, 26], [2748, 60], [2749, 136], [2750, 200], [4066, 22], [4067, 14], [4068, 73], [4069, 20], [4073, 26], [4310, 26], [4311, 14], [4312, 26], [4313, 46], [4314, 32], [4630, 20], [4631, 14], [4632, 14], [4633, 14], [4634, 18], [4635, 34], [4641, 26], [4642, 40], [4643, 24], [4674, 16], [4675, 48], [4676, 34], [4840, 8]]
2017-04-11 15:24:36,910 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4559
2017-04-11 15:24:36,910 TextArray#__init__ start
2017-04-11 15:24:38,730 TextArray#to_vec start
2017-04-11 15:24:38,785 TextArray#to_vec end
2017-04-11 15:24:38,785 use algorithm: Multi-layer Perceptron
2017-04-11 15:24:38,785 learning_parameter has not parameter [activation]
2017-04-11 15:26:18,451 best_params_: {'activation': 'identity'}
2017-04-11 15:26:18,451 after Bot#__get_estimator
2017-04-11 15:26:18,485 before Evaluator#evaluate
2017-04-11 15:26:18,485 self.threshold: 0.0
2017-04-11 15:26:24,721 Evaluator#evaluate#elapsed time: 6236.170053 ms
2017-04-11 15:26:24,722 accuracy: 0.97458369851
2017-04-11 15:26:24,722 end Bot#learn
2017-04-11 15:26:24,722 Bot.learn processing time: 109.64664506912231 [sec]
2017-04-11 15:26:24,746 TextArray#__init__ start
2017-04-11 15:26:24,748 Reply#predict text_array.separated_sentences: ['入会 する']
2017-04-11 15:26:24,748 TextArray#to_vec start
2017-04-11 15:26:24,749 TextArray#to_vec end
2017-04-11 15:26:24,749 Reply#predict features:   (0, 88)	0.707106781187
  (0, 266)	0.707106781187
2017-04-11 15:26:24,753 learning_parameter: {'_include_failed_data': False, '_params_for_algorithm': {}, '_include_tag_vector': False, '_algorithm': 0, '_classify_threshold': None}
2017-04-11 15:26:24,753 MyopeServer.learn processing time: 0.00025010108947753906 [sec]
```